In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from preprocessy.pipelines import Preprocessy

In [3]:
def test_input():
    params = {'target_label':"Survived",'cat_cols':['Survived','Pclass','Sex','Parch','Embarked'],'drop':True,'column_list':["PassengerId","Name","Ticket","Cabin"]}
    p = Preprocessy(train_df_path="/Users/mac/preprocessy/datasets/titanic.csv",test_df_path="/Users/mac/preprocessy/datasets/titanic.csv", params=params)
    p.process()
    return params
    
    

In [4]:
params = test_input()

-------------Initializing pipeline-------------
-------------Completed reading and loading the .csv file in 0:00:00.050987 -------------
-------------Completed calculating the categorical columns in 0:00:00.002801 -------------
-------------Completed handling the null values in dataframe in 0:00:00.049393 -------------
-------------Completed encoding categorical columns in 0:00:00.030740 ---------------
-------------Completed handling outliers in 0:00:00.032897 -------------
-------------Completed scaling values in dataframe in 0:00:00.000951 -------------
/Users/mac/preprocessy/preprocessy/parse/_summarize.py:31: UserWarning: Parameter 'target_label' is empty. If not provided and is present in dataframe, it may get encoded. To mitigate, provide the target_label from dataframe or provide explicit list of columns for encoding via the 'cat_cols' parameter
  warnings.warn(
/Users/mac/preprocessy/preprocessy/encoding/_categorical.py:51: UserWarning: target_label may get encoded. Please rem

In [23]:
train_new = params["X_train"]
test_new = params["X_test"]
train_y = train_new["Survived"]
test_y = test_new["Survived"]
colstodrop = ["Survived","Sex","Parch","SurvivedEncoded","PclassEncoded","Embarked"]
train_new.drop(colstodrop,axis=1,inplace=True)
test_new.drop(colstodrop,axis=1,inplace=True)

In [12]:

import pandas as pd
from sklearn.model_selection import train_test_split
original = pd.read_csv("/Users/mac/preprocessy/datasets/titanic.csv")
cols = ["PassengerId","Name","Ticket","Cabin"]
original.drop(cols,axis=1,inplace=True)
original.dropna(inplace=True)
original["SexEncoded"] = original["Sex"].apply(lambda x: 0 if x=='female' else 1)
original["EmbarkedEncoded"] = original["Embarked"].apply(lambda x: 0 if x=='C' else( 1 if x=='S' else 2))
original.drop(['Sex','Embarked'],axis=1,inplace=True)
y = original["Survived"]
X = original.drop(["Survived"],axis=1)
trainx, testx, trainy, testy = train_test_split(X,y,test_size=0.2)


In [8]:
from sklearn.linear_model import LogisticRegression


In [24]:
l = LogisticRegression(max_iter=200)
l.fit(train_new,train_y)

LogisticRegression(max_iter=200)

In [26]:
print("model score: %.3f" % l.score(test_new, test_y))

model score: 0.817


In [20]:
def sc(X,y):
    trainx, testx, trainy, testy = train_test_split(X,y,test_size=0.2)
    l = LogisticRegression(max_iter=200)
    l.fit(trainx,trainy)
    t = l.score(testx, testy)
    return t

In [21]:
z = []
for _ in range(5):
    z.append(sc(X,y))
sum(z)/len(z)


0.793006993006993

In [19]:
z

[0.8391608391608392,
 0.8391608391608392,
 0.8391608391608392,
 0.8391608391608392,
 0.8391608391608392]